In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
dates = ["2023-05-09", "2023-05-10","2023-05-11","2023-05-23","2023-05-24","2023-07-04","2023-07-05","2023-07-06",\
         "2023-07-18","2023-07-20","2023-08-01","2023-08-02","2023-08-03","2023-08-15","2023-08-16","2023-08-17",\
         "2023-08-29","2023-09-12"]
pdf = PdfPages("/home/craiglagegit/DATA/Shutter_State_Multiple.pdf")

for date in dates:
    start = Time(f"{date}T16:00:00Z", scale='utc')
    date_split = date.split('-')
    end_date = f"{date_split[0]}-{date_split[1]}-{int(date_split[2])+1:02}"
    print(date, end_date)
    end = Time(f"{end_date}T10:00:00Z", scale='utc')
    try:
        state = await client.select_time_series('lsst.sal.ATDome.logevent_mainDoorState', \
                                            ['*'],  start, end)
        shut = await client.select_time_series('lsst.sal.ATDome.position', \
                                                    ['mainDoorOpeningPercentage'],  start, end)
        close_command = await client.select_time_series('lsst.sal.ATDome.command_closeShutter', \
                                                    ['*'],  start, end)
        print(len(state), len(shut), len(close_command))
        if len(shut) == 0:
               continue
        fig, ax = plt.subplots(1,1, figsize=(15,5))
        ax.set_title(f"Shutter_State_{date}", fontsize=24)
        state['state'].plot(ax=ax, color='blue', label='Actual State')
        ax.set_yticks([1,2,3,4,5,6,7])
        ax.set_yticklabels(['Closed', 'Opened', 'Ajar', 'Opening', 'Closing','',''])
        ax.set_ylabel("Door State")
        ax1 = ax.twinx()
        shut['mainDoorOpeningPercentage'].plot(ax=ax1, color='red', label='Shutter Open Percentage')
        ax1.set_ylabel("Open percentage")
        ax1.set_ylim(0,115)
        for close in close_command.index:
            ax1.axvline(close, ls='--', color='green', label='Close Command')
        ax.legend(loc='lower left')
        ax1.legend(loc='upper left')
        pdf.savefig(fig)  # saves the current figure into a pdf page
        plt.clf()
    except:
        continue
pdf.close()
